In [212]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import datetime
import collections
import os
import numpy as np
import _levenshtein as lev
import tensorflow as tf


In [213]:
# Utility functions for reading files

def data_read_words(filename):
  with tf.gfile.GFile(filename, "r") as f:
    return f.read().replace("\n", "<eos>").split()


def data_build_vocab(filename):
  data = data_read_words(filename)

  counter = collections.Counter(data)
  count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

  words, _ = list(zip(*count_pairs))
  word_to_id = dict(zip(words, range(len(words))))

  return word_to_id


def data_file_to_word_ids(filename, word_to_id):
  data = data_read_words(filename)
  return [word_to_id[word] for word in data]


def load_raw_data(data_path=None):
  """Load raw data from data directory "data_path".

  Reads text files, converts strings to integer ids,
  and performs mini-batching of the inputs.

  """

  train_path = os.path.join(data_path, "data.train.txt") 
  valid_path = os.path.join(data_path, "data.valid.txt")
  test_path = os.path.join(data_path, "data.test.txt")

  word_to_id = data_build_vocab(train_path)
  train_data = data_file_to_word_ids(train_path, word_to_id)
  valid_data = data_file_to_word_ids(valid_path, word_to_id)
  test_data = data_file_to_word_ids(test_path, word_to_id)
  vocabulary = len(word_to_id)
  return train_data, valid_data, test_data, vocabulary


def data_iterator(raw_data, batch_size, num_steps):
  """Iterate on the raw data.

  This generates batch_size pointers into the raw PTB data, and allows
  minibatch iteration along these pointers.

  Args:
    raw_data: one of the raw data outputs from ptb_raw_data.
    batch_size: int, the batch size.
    num_steps: int, the number of unrolls.

  Yields:
    Pairs of the batched data, each a matrix of shape [batch_size, num_steps].
    The second element of the tuple is the same data time-shifted to the
    right by one.

  Raises:
    ValueError: if batch_size or num_steps are too high.
  """
  raw_data = np.array(raw_data, dtype=np.int32)

  data_len = len(raw_data)
  batch_len = data_len // batch_size
  data = np.zeros([batch_size, batch_len], dtype=np.int32)
  for i in range(batch_size):
    data[i] = raw_data[batch_len * i:batch_len * (i + 1)]

  epoch_size = (batch_len - 1) // num_steps

  if epoch_size == 0:
    raise ValueError("epoch_size == 0, decrease batch_size or num_steps")

  for i in range(epoch_size):
    x = data[:, i*num_steps:(i+1)*num_steps]
    y = data[:, i*num_steps+1:(i+1)*num_steps+1]
    yield (x, y)


In [214]:
class VanillaRNN(object):
  num_layers = 1

class TwoLayerRNN(object):
  num_layers = 2

class BidirectionalRNN(object):
  num_layers = 2

class SmallConfig(object):
  """Small config."""
  init_scale = 0.1
  learning_rate = 1.0
  max_grad_norm = 5  
  num_steps = 20
  hidden_size = 100
  max_epoch = 4
  max_max_epoch = 13
  keep_prob = 1.0
  lr_decay = 0.5
  batch_size = 20
  vocab_size = 10000


class MediumConfig(object):
  """Medium config."""
  init_scale = 0.05
  learning_rate = 1.0
  max_grad_norm = 5
  num_steps = 35
  hidden_size = 650
  max_epoch = 6
  max_max_epoch = 39
  keep_prob = 0.5
  lr_decay = 0.8
  batch_size = 20
  vocab_size = 10000


class LargeConfig(object):
  """Large config."""
  init_scale = 0.04
  learning_rate = 1.0
  max_grad_norm = 10
  num_steps = 35
  hidden_size = 1500
  max_epoch = 14
  max_max_epoch = 55
  keep_prob = 0.35
  lr_decay = 1 / 1.15
  batch_size = 20
  vocab_size = 10000


class TestConfig(object):
  """Tiny config, for testing."""
  init_scale = 0.1
  learning_rate = 1.0
  max_grad_norm = 1
  num_layers = 1
  num_steps = 2
  hidden_size = 2
  max_epoch = 1
  max_max_epoch = 1
  keep_prob = 1.0
  lr_decay = 0.5
  batch_size = 20
  vocab_size = 10000



In [215]:
class LSTM_DropoutModel(object):
  """The LSTM_DropoutModel model."""

#------------------------------------------------------------------------
  def __init__(self, is_training, config, model):
#------------------------------------------------------------------------
    self.batch_size = batch_size = config.batch_size
    self.num_steps = num_steps = config.num_steps
    size = config.hidden_size
    vocab_size = config.vocab_size

    self._input_data = tf.placeholder(tf.int32, [batch_size, num_steps])
    self._targets = tf.placeholder(tf.int32, [batch_size, num_steps])

    with tf.device("/cpu:0"): # use primary CPU; to use GPU - "/gpu:0"
      embedding = tf.get_variable("embedding", [vocab_size, size])
      inputs = tf.nn.embedding_lookup(embedding, self._input_data)

    #if is_training and config.keep_prob < 1:
      #inputs = tf.nn.dropout(inputs, config.keep_prob)

    if FLAGS_model=="BidirectionalRNN":
        # forward direction cell
        lstm_cell_fw = tf.nn.rnn_cell.BasicLSTMCell(size, forget_bias=0.0)
        # backward direction cell
        lstm_cell_bw = tf.nn.rnn_cell.BasicLSTMCell(size, forget_bias=0.0)
        outputs, _, _ = rnn.bidirectional_rnn(lstm_cell_fw, lstm_cell_bw, x, dtype=tf.float32)
    else:    
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(size, forget_bias=0.0)
  
        # using Dropout
        if is_training and config.keep_prob < 1:
          lstm_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_cell, output_keep_prob=config.keep_prob)

        cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell] * model.num_layers)
        self._initial_state = cell.zero_state(batch_size, tf.float32)

    outputs = []
    state = self._initial_state
    with tf.variable_scope("RNN"):
      for time_step in range(num_steps):
        if time_step > 0: 
            tf.get_variable_scope().reuse_variables()
        (cell_output, state) = cell(inputs[:, time_step, :], state)
        outputs.append(cell_output)

    output = tf.reshape(tf.concat(1, outputs), [-1, size])  
    softmax_w = tf.get_variable("softmax_w", [size, vocab_size])
    softmax_b = tf.get_variable("softmax_b", [vocab_size])
    self.logits = logits = tf.matmul(output, softmax_w) + softmax_b
    
    #.sequence_loss_by_example([logits], [targets], [weights]) 
    loss = tf.nn.seq2seq.sequence_loss_by_example([logits], 
                                                  [tf.reshape(self._targets, [-1])],
                                                  [tf.ones([batch_size * num_steps])])
    self._cost = cost = tf.reduce_sum(loss) / batch_size 
    self._final_state = state

    if not is_training:
      return

    self._lr = tf.Variable(0.0, trainable=False)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),
                                      config.max_grad_norm)
    optimizer = tf.train.GradientDescentOptimizer(self.lr)
    self._train_op = optimizer.apply_gradients(zip(grads, tvars))
    
#------------------------------------------------------------------------
  def assign_lr(self, session, lr_value):
#------------------------------------------------------------------------
    session.run(tf.assign(self.lr, lr_value))

  @property
  def input_data(self):
    return self._input_data

  @property
  def targets(self):
    return self._targets

  @property
  def initial_state(self):
    return self._initial_state

  @property
  def cost(self):
    return self._cost

  @property
  def final_state(self):
    return self._final_state

  @property
  def lr(self):
    return self._lr

  @property
  def train_op(self):
    return self._train_op
#------------------------------------------------------------------------


In [216]:
#------------------------------------------------------------------------
def run_epoch(session, m, data, eval_op, verbose=False):
#------------------------------------------------------------------------
  """Runs the model on the given data."""
  epoch_size = ((len(data) // m.batch_size) - 1) // m.num_steps
  start_time = time.time()
  costs = 0.0
  iters = 0
  levenshtein_distance = 0.0

  state = m.initial_state.eval()
  for step, (x, y) in enumerate(data_iterator(data, m.batch_size,m.num_steps)):
    # run the loop
    cost, state, _ = session.run([m.cost, m.final_state, eval_op],
                                 {m.input_data: x,
                                  m.targets: y,
                                  m.initial_state: state})
    costs += cost
    iters += m.num_steps

    if verbose and step % (epoch_size // 10) == 10:
      print("%.3f perplexity: %.3f speed: %.0f wps" %
            (step * 1.0 / epoch_size, np.exp(costs / iters),
             iters * m.batch_size / (time.time() - start_time)))
  perplexity = np.exp(costs / iters)
  return perplexity


In [217]:
#------------------------------------------------------------------------
def get_config():
#------------------------------------------------------------------------
  if FLAGS_config == "small":
    return SmallConfig()
  elif FLAGS_config == "medium":
    return MediumConfig()
  elif FLAGS_config == "large":
    return LargeConfig()
  elif FLAGS_config == "test":
    return TestConfig()
  else:
    raise ValueError("Invalid Configuration: %s", FLAGS_config)

#------------------------------------------------------------------------
def get_model():
#------------------------------------------------------------------------
  if FLAGS_model == "VanillaRNN":
    return VanillaRNN()
  elif FLAGS_model == "TwoLayerRNN":
    return TwoLayerRNN()
  elif FLAGS_model == "BidirectionalRNN":
    return BidirectionalRNN()
  else:
    raise ValueError("%s :Model Not Supported.", FLAGS_model)


In [218]:
#------------------------------------------------------------------------
def main(_):
#------------------------------------------------------------------------
  starttime = datetime.datetime.now()
  print(starttime.strftime("%A, %d. %B %Y %I:%M%p"))
  if not FLAGS_data_path:
    raise ValueError("Must set --data_path to the data directory")

  raw_data = load_raw_data(FLAGS_data_path)
  train_data, valid_data, test_data, _ = raw_data

  model = get_model()
  config = get_config()
  eval_config = get_config()
  eval_config.batch_size = 1
  eval_config.num_steps = 1

  with tf.Graph().as_default(), tf.Session() as session:
    initializer = tf.random_uniform_initializer(-config.init_scale,
                                                config.init_scale)
    with tf.variable_scope("model", reuse=None, initializer=initializer):
      m = LSTM_DropoutModel(is_training=True, config=config, model=model)
    with tf.variable_scope("model", reuse=True, initializer=initializer):
      mvalid = LSTM_DropoutModel(is_training=False, config=config, model=model)
      mtest = LSTM_DropoutModel(is_training=False, config=eval_config, model=model)

    tf.initialize_all_variables().run()

    for i in range(config.max_max_epoch):  # run for number of epochs
      lr_decay = config.lr_decay ** max(i - config.max_epoch, 0.0)
      m.assign_lr(session, config.learning_rate * lr_decay)

      print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
      train_perplexity = run_epoch(session, m, train_data, m.train_op, verbose=True)
      print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))
      valid_perplexity = run_epoch(session, mvalid, valid_data, tf.no_op())
      print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, valid_perplexity))

    test_perplexity = run_epoch(session, mtest, test_data, tf.no_op())
    print("Test Perplexity: %.3f" % test_perplexity)
    endtime = datetime.datetime.now()
    print(endtime.strftime("%A, %d. %B %Y %I:%M%p"))
    elapsedTime = endtime - starttime
    print (elapsedTime)

    print("Done")



In [219]:
#------------------------------------------------------------------------
if __name__ == "__main__":
#------------------------------------------------------------------------

  FLAGS_config="small" #choices are: small, medium, large
  FLAGS_model="VanillaRNN" # choices are: TwoLayerRNN, VanillaRNN, BidirectionalRNN
  FLAGS_data_path="data/"
  print('VanillaRNN - small config - with Dropout')

  tf.app.run()  


VanillaRNN - small config - with Dropout
Wednesday, 29. June 2016 06:33AM
Epoch: 1 Learning rate: 1.000
0.004 perplexity: 3551.483 speed: 2434 wps
0.104 perplexity: 723.470 speed: 2588 wps
0.204 perplexity: 537.804 speed: 2596 wps
0.304 perplexity: 442.961 speed: 2560 wps
0.404 perplexity: 389.204 speed: 2554 wps
0.504 perplexity: 353.777 speed: 2538 wps
0.604 perplexity: 322.626 speed: 2545 wps
0.703 perplexity: 301.225 speed: 2550 wps
0.803 perplexity: 284.143 speed: 2548 wps
0.903 perplexity: 268.191 speed: 2550 wps
Epoch: 1 Train Perplexity: 256.158
Epoch: 1 Valid Perplexity: 179.020
Epoch: 2 Learning rate: 1.000
0.004 perplexity: 209.244 speed: 2402 wps
0.104 perplexity: 153.880 speed: 2566 wps
0.204 perplexity: 160.816 speed: 2570 wps
0.304 perplexity: 156.384 speed: 2554 wps
0.404 perplexity: 153.725 speed: 2547 wps
0.504 perplexity: 151.614 speed: 2556 wps
0.604 perplexity: 147.528 speed: 2555 wps
0.703 perplexity: 145.713 speed: 2561 wps
0.803 perplexity: 144.021 speed: 2557 w

SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.
